In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import classification_report, log_loss
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import pandas as pd
import numpy as np
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import LearningRateScheduler
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from keras import optimizers
import random
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
import datetime
import numpy as np
import os, tempfile
import pandas as pd
import re
from sklearn import preprocessing
import itertools
from scipy import linalg
from sklearn import svm
import tensorflow as tf
from sklearn.model_selection import cross_val_score,cross_val_predict
import time
from sklearn.metrics import mean_squared_error, r2_score
pd.options.display.max_rows = 20
pd.options.display.max_columns = 120
from sklearn import metrics
from sklearn import model_selection
from tensorflow.contrib import learn
from sklearn import cross_validation
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from mpl_toolkits.mplot3d import Axes3D
import math
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
random.seed(12345)
from keras.layers import Dropout
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from keras.layers import Bidirectional
pd.options.display.max_columns = 100
from keras.layers import Activation, Dense, GRU

In [ ]:
path = './'
name = ''
out_name = path + name + 'submission_py.csv'

In [ ]:

import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          doprint=True,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    if doprint:
        print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    if doprint:
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# read data
train_data = pd.read_csv('train.csv')

In [ ]:
train = train_data

In [ ]:
train = train.drop(['Unnamed: 0','Time_0','Time_1','Time_2','Time_3',
                    'Time_4','Time_5','Time_6','Time_7','Time_8','Time_9','Time_10','Time_11',
                   'Time_12','Time_13','Time_14','Time_15','Time_16','Time_17','Time_18','Time_19',
                   'Time_20','Time_21','Time_22','Time_23','Time_24','Time_25','Time_26','Time_27',
                   'Time_28','Time_29'],axis=1)

In [ ]:
qq = list(range(0,180))
X = train.iloc[:,qq].as_matrix()
y = train.iloc[:,-1].as_matrix()

In [ ]:
X = np.nan_to_num(X)

In [ ]:
X.shape

In [ ]:
X_reshaped = X.reshape(1724760,3)
poly = PolynomialFeatures(2)
polyx = poly.fit_transform(X_reshaped)

In [ ]:
polyx = polyx.reshape(28746,600)

In [ ]:
scaler = preprocessing.StandardScaler(copy=False, with_mean=True, with_std=True)
scaler = scaler.fit(polyx)
# normalize the dataset and print
normalized = scaler.transform(polyx)
# inverse transform and print

In [ ]:
X_new = normalized.reshape(28746,30,20)

In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
X_train, X_val, y_train, y_val = train_test_split(X_new, dummy_y, test_size=0.05)

In [ ]:
print(X_train.shape)
print(X_val.shape)

In [ ]:
df = pd.DataFrame(y,columns=[['y']])
df = pd.DataFrame(df.groupby(['y'])['y'].count())
df['freq'] = round((df['y']/df['y'].sum()),2)

In [ ]:
df.index = df.index -1 
weight = df.freq.to_dict()


In [ ]:
# design network
model = Sequential()
model.add(Bidirectional(LSTM(4, return_sequences=True), input_shape=(X_train.shape[1] ,X_train.shape[2])))
model.add(Bidirectional(LSTM(8, return_sequences=True)))
model.add(Bidirectional(LSTM(16, return_sequences=True)))
model.add(Bidirectional(LSTM(32, return_sequences=False)))

model.add(Dropout(0.5))


model.add(Dense(y_train.shape[1],activation='softmax'))#Create output layer

# Compile model

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])


In [ ]:
%%time
epochs = [16,32,64]
batch_size = [64,128,256,512,1024,2048,4096,8192]

# fit network
for j in epochs:
    for i in batch_size:
        model.fit(X_train, y_train, epochs=j, batch_size=i,validation_data=(X_val, y_val), verbose=1, shuffle=False,class_weight=weight)

In [ ]:
X_train.shape

In [ ]:
pred_proba = model.predict_proba(X_val)
pred_ = model.predict_classes(X_val)

In [ ]:
cnf_matrix = confusion_matrix(y_val.argmax(axis=1), pred_)
plt.figure()
classes = range(0,25)
plot_confusion_matrix(cnf_matrix, classes, normalize=True, doprint=False, title='Normalized confusion matrix')

In [ ]:
print(classification_report(y_pred=pred_,y_true=y_val.argmax(axis=1)))
print('log_loss: {}'.format(log_loss(y_pred=pred_proba,y_true=y_val.argmax(axis=1))))

## Test

In [ ]:
test_data = pd.read_csv(path + name + 'test.csv')
test_data = test_data.drop(['Unnamed: 0','Time_0','Time_1','Time_2','Time_3',
                    'Time_4','Time_5','Time_6','Time_7','Time_8','Time_9','Time_10','Time_11',
                   'Time_12','Time_13','Time_14','Time_15','Time_16','Time_17','Time_18','Time_19',
                   'Time_20','Time_21','Time_22','Time_23','Time_24','Time_25','Time_26','Time_27',
                   'Time_28','Time_29'],axis=1)
test = np.nan_to_num(x=test_data)

In [ ]:
test.shape

In [ ]:
test.shape[0]*test.shape[1]

In [ ]:
(test.shape[0]*test.shape[1])/3

In [ ]:
test_reshaped = test.reshape(16275060,3)

In [ ]:
polyxtest = poly.fit_transform(test_reshaped)

In [ ]:
polyxtest = polyxtest.reshape(271251,600)

In [ ]:
normalized_test = scaler.transform(polyxtest)
normalized_test = normalized_test.reshape(271251,30,20)

In [ ]:
pred = model.predict_classes(normalized_test)

In [ ]:
pred

In [ ]:
pred = pred.astype(int)

In [ ]:
df = pd.DataFrame(pred)
df = df+1
df.to_csv(out_name, header=False)

In [ ]:
#compress result file
import gzip

in_data = open(out_name, "rb").read()
out_gz = out_name+".gz"
gzf = gzip.open(out_gz, "wb")
gzf.write(in_data)
gzf.close()

In [ ]:
df.to_csv(out_name, header=False)

In [ ]:
in_data = open(out_name, "rb").read()
out_gz = out_name+".gz"
gzf = gzip.open(out_gz, "wb")
gzf.write(in_data)
gzf.close()

In [ ]:
df.max()

In [ ]:
pred.max()